In [ ]:
# =============================================================
# PUNTO 3 – Métricas internas (NOMBRE DEL PORTAL) – 
# - Se debe usar y trabajar con salida Punto 1 archivo 
# - Normaliza tipos, limpia listas, calcula scores y niveles
# - portal_maturity_level: bajo [0-30), medio [30-70), alto [70-100]
# =============================================================

import pandas as pd
import numpy as np
import re
from datetime import datetime

# -------------------------------------------------------------
# 1) CONFIGURACION
# -------------------------------------------------------------
FILE = "Punto1_Ayto_Barcelona_v2 DEFINITIVO.xlsx"   # <-- tu input Punto 1
OUT_DATA_XLSX = "Punto3_NOMBREDELARCHIVO_USARPARARESULTADOS.xlsx"
OUT_DATA_CSV  = "Punto3_NOMBREDELARCHIVO_USARPARARESULTADOS.csv"
OUT_SUM_XLSX  = "Punto3_NOMBREDELARCHIVO_COMPLEMENTO.xlsx"
OUT_SUM_CSV   = "Punto3_NOMBREDELARCHIVO_COMPLEMENTO.csv"

df = pd.read_excel(FILE)

print("Dataset cargado:", df.shape)
print("Columnas detectadas:", df.columns.tolist())

TODAY = pd.to_datetime(datetime.now().date())

# -------------------------------------------------------------
# 2) HELPERS
# -------------------------------------------------------------
def to_int01(s):
    """Convierte valores a 0/1 de forma robusta."""
    x = pd.to_numeric(s, errors="coerce")
    x = x.fillna(0)
    return (x > 0).astype(int)

def is_nonempty_text(s):
    """True si hay texto no vacío (evita 'nan', None, '')."""
    if pd.isna(s):
        return 0
    t = str(s).strip()
    if t == "" or t.lower() in {"nan", "none", "null"}:
        return 0
    return 1

def months_since(date):
    """Meses enteros desde date hasta TODAY."""
    if pd.isna(date):
        return np.nan
    return (TODAY.year - date.year) * 12 + (TODAY.month - date.month)

def normalize_str(x):
    return "" if pd.isna(x) else str(x).strip()

# -------------------------------------------------------------
# 3) NORMALIZACIÓN FECHAS (issued / modified)
# -------------------------------------------------------------
for c in ["issued", "modified"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

df["age_months_modified"] = df["modified"].apply(months_since) if "modified" in df.columns else np.nan
df["age_month_issued"]    = df["issued"].apply(months_since) if "issued" in df.columns else np.nan

# -------------------------------------------------------------
# 4) FORMATOS -> listas + has_open_format
# -------------------------------------------------------------
OPEN_FORMATS = {
    "CSV","JSON","GEOJSON","XML","RDF","TTL","TURTLE","N-TRIPLES","NT","JSON-LD","JSONLD"
}

def normalize_formats(fmt_original):
    if pd.isna(fmt_original) or str(fmt_original).strip() == "":
        return []
    raw = str(fmt_original).strip().upper()
    parts = re.split(r"[;,|\n\r\t]+", raw)

    out = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        p = p.replace(" ", "")
        if "JSONLD" in p or "JSON-LD" in p:
            out.append("JSON-LD")
        elif "N-TRIPLES" in p or p == "NTRIPLES":
            out.append("N-TRIPLES")
        else:
            out.append(p)

    # unique preservando orden
    seen, uniq = set(), []
    for x in out:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq

def split_open_nonopen(tokens):
    open_list, nonopen_list = [], []
    for t in tokens:
        (open_list if t in OPEN_FORMATS else nonopen_list).append(t)
    return open_list, nonopen_list

if "format" not in df.columns:
    df["format"] = np.nan

df["formats_normalized"] = df["format"].apply(normalize_formats)
tmp = df["formats_normalized"].apply(split_open_nonopen)
df["open_formats_list"] = tmp.apply(lambda x: x[0])
df["non_open_formats_list"] = tmp.apply(lambda x: x[1])

df["open_format_count"] = df["open_formats_list"].apply(len)
df["non_open_format_count"] = df["non_open_formats_list"].apply(len)
df["has_open_format"] = (df["open_format_count"] > 0).astype(int)

# Si Punto 1 trae has_allowed_format, lo respetamos; si no, aproximamos
if "has_allowed_format" not in df.columns:
    df["has_allowed_format"] = df["has_open_format"]
df["has_allowed_format"] = to_int01(df["has_allowed_format"])

# -------------------------------------------------------------
# 5) LICENCIA -> license_present + license_open
# -------------------------------------------------------------
if "license" not in df.columns:
    df["license"] = np.nan

if "license_present" not in df.columns:
    df["license_present"] = df["license"].notna().astype(int)
df["license_present"] = to_int01(df["license_present"])

if "license_open" not in df.columns:
    df["license_open"] = 0
df["license_open"] = to_int01(df["license_open"])

# -------------------------------------------------------------
# 6) DOWNLOAD URL -> download_url_present + lista unificada
# -------------------------------------------------------------
def parse_download_urls(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, (list, tuple, set)):
        urls = [str(u).strip() for u in x if str(u).strip().lower().startswith("http")]
        return list(dict.fromkeys(urls))
    s = str(x).strip()
    if s == "":
        return []
    parts = re.split(r"[,\s]+", s)
    urls = [p for p in parts if p.lower().startswith("http")]
    return list(dict.fromkeys(urls))

urls_1 = df["download_url"].apply(parse_download_urls) if "download_url" in df.columns else pd.Series([[]]*len(df))
urls_2 = df["download_urls"].apply(parse_download_urls) if "download_urls" in df.columns else pd.Series([[]]*len(df))

df["download_urls_list"] = [
    list(dict.fromkeys((a or []) + (b or [])))
    for a, b in zip(urls_1.tolist(), urls_2.tolist())
]
df["download_url_present"] = (df["download_urls_list"].apply(len) > 0).astype(int)

# -------------------------------------------------------------
# 7) update_frequency -> update_freq_months
# -------------------------------------------------------------
if "update_frequency" not in df.columns:
    df["update_frequency"] = np.nan

def map_update_frequency_to_months(freq):
    if pd.isna(freq) or str(freq).strip() == "":
        return np.nan

    f = str(freq).strip().lower().replace("_", " ").replace("-", " ").strip()

    # equivalencias ES/EN
    if f in ["monthly", "mensual", "mensualmente"]:
        return 1.0
    if f in ["weekly", "semanal", "semanalmente"]:
        return 0.25
    if f in ["daily", "diaria", "diario"]:
        return 1.0 / 30.0
    if f in ["quarterly", "trimestral", "trimestralmente"]:
        return 3.0
    if f in ["semiannual", "semestral", "semestralmente"]:
        return 6.0
    if f in ["annual", "anual", "anualmente", "yearly"]:
        return 12.0
    if f in ["biennial", "bianual", "bienal", "cada 2 años", "cada 2 anos"]:
        return 24.0

    if f in ["no definido", "indefinido", "irregular", "no definida", "unknown", "desconocido"]:
        return np.nan

    return np.nan

df["update_freq_months"] = df["update_frequency"].apply(map_update_frequency_to_months)

# -------------------------------------------------------------
# 8) traceable_origen = publisher + dataset_uri + identifier
# -------------------------------------------------------------
# Barcelona ya trae publisher/dataset_uri/identifier; si faltara alguno, se tolera.
for col in ["publisher", "dataset_uri", "identifier"]:
    if col not in df.columns:
        df[col] = np.nan

df["_publisher_present"] = df["publisher"].apply(is_nonempty_text).astype(int)
df["_dataset_uri_present"] = df["dataset_uri"].apply(is_nonempty_text).astype(int)
df["_identifier_present"] = df["identifier"].apply(is_nonempty_text).astype(int)

df["traceable_origen"] = (
    (df["_publisher_present"] == 1) &
    (df["_dataset_uri_present"] == 1) &
    (df["_identifier_present"] == 1)
).astype(int)

# -------------------------------------------------------------
# 9) traceable_temporal = (modified + update_frequency informada)
# -------------------------------------------------------------
df["_has_age_modified"] = df["age_months_modified"].notna().astype(int)
df["_has_age_issued"] = df["age_month_issued"].notna().astype(int)
df["_has_update_freq"] = df["update_frequency"].notna().astype(int)

df["traceable_temporal"] = (
    (df["_has_age_modified"] == 1) &
    (df["_has_age_issued"] == 1) &
    (df["_has_update_freq"] == 1)
).astype(int)

# -------------------------------------------------------------
# 10) traceability_score = (origen + temporal + reutilizable/doi)/3 * 100
# -------------------------------------------------------------
if "has_doi" in df.columns:
    df["traceable_reutilizable"] = to_int01(df["has_doi"])
elif "doi" in df.columns:
    df["traceable_reutilizable"] = df["doi"].notna().astype(int)
else:
    df["traceable_reutilizable"] = 0

df["traceability_score"] = (
    (df["traceable_origen"] + df["traceable_temporal"] + df["traceable_reutilizable"]) / 3.0 * 100
).round(2)

# -------------------------------------------------------------
# 11) interoperability_semantics = (dcat + api_type + vocab + semantic_serial)/4
# -------------------------------------------------------------
if "portal_supports_dcat_dcatap" not in df.columns:
    df["portal_supports_dcat_dcatap"] = 0
if "api_type" not in df.columns:
    df["api_type"] = np.nan
if "uses_controlled_vocab" not in df.columns:
    df["uses_controlled_vocab"] = 0
if "has_semantic_serialization" not in df.columns:
    df["has_semantic_serialization"] = 0

df["portal_supports_dcat_dcatap"] = to_int01(df["portal_supports_dcat_dcatap"])
df["uses_controlled_vocab"] = to_int01(df["uses_controlled_vocab"])
df["has_semantic_serialization"] = to_int01(df["has_semantic_serialization"])
df["_api_type_present"] = df["api_type"].apply(is_nonempty_text).astype(int)

df["interoperability_semantics"] = (
    (df["portal_supports_dcat_dcatap"] + df["_api_type_present"] + df["uses_controlled_vocab"] + df["has_semantic_serialization"])
    / 4.0 * 100
).round(2)

# -------------------------------------------------------------
# 12) interoperability_technical = (license_open + has_open_format)/2
# -------------------------------------------------------------
df["interoperability_technical"] = (
    (df["license_open"] + df["has_open_format"]) / 2.0 * 100
).round(2)

# -------------------------------------------------------------
# 13) accessibility_score = (api_rest + allowed_format + license_present + download_url_present)/4
# -------------------------------------------------------------
if "portal_has_api_rest" not in df.columns:
    df["portal_has_api_rest"] = 0
df["portal_has_api_rest"] = to_int01(df["portal_has_api_rest"])

df["accessibility_score"] = (
    (df["portal_has_api_rest"] + df["has_allowed_format"] + df["license_present"] + df["download_url_present"])
    / 4.0 * 100
).round(2)

# -------------------------------------------------------------
# 14) quality_score = (has_data_dictionary + description_present)/2
# -------------------------------------------------------------
if "has_data_dictionary" not in df.columns:
    df["has_data_dictionary"] = 0
df["has_data_dictionary"] = to_int01(df["has_data_dictionary"])

if "description" not in df.columns:
    df["description"] = np.nan
df["_description_present"] = df["description"].apply(is_nonempty_text).astype(int)

df["quality_score"] = (
    (df["has_data_dictionary"] + df["_description_present"]) / 2.0 * 100
).round(2)

# -------------------------------------------------------------
# 15) portal_maturity = promedio de 5 scores
# -------------------------------------------------------------
df["portal_maturity"] = (
    (df["accessibility_score"] + df["interoperability_semantics"] + df["interoperability_technical"] + df["traceability_score"] + df["quality_score"])
    / 5.0
).round(2)

# NUEVOS CORTES 
df["portal_maturity_level"] = pd.cut(
    df["portal_maturity"],
    bins=[-0.1, 30, 60, 100],
    labels=["bajo", "medio", "alto"]
).astype(str)

# -------------------------------------------------------------
# 16) LIMPIEZA columnas viejas/auxiliares
# -------------------------------------------------------------
COLUMNS_TO_DROP = [
    "frequency_documented",
    "selected_for_analysis",
    "age_since_issued_months",
    "months_since_modified",
    "age_reference_date",
    "months_since_reference",
    "stale_12m",
    "license_class",
    "license_family",
    "update_cadence_group",
    "updates_slower_than_month",
    "api_score",
    "freq_score",
    "operational_traceability",
    "update_transparency_score",
    "license_openness_score",
    "format_openness_score",
    "metadata_completeness",
    "maturity_index",
    "maturity_level",
    "format_openness_score",
]

df.drop(columns=[c for c in COLUMNS_TO_DROP if c in df.columns], inplace=True)

# columnas auxiliares internas que empiezan por "_"
aux_cols = [c for c in df.columns if c.startswith("_")]
df.drop(columns=aux_cols, inplace=True, errors="ignore")

# -------------------------------------------------------------
# 17) EXPORT DATASET-LEVEL
# -------------------------------------------------------------
df.to_excel(OUT_DATA_XLSX, index=False)
df.to_csv(OUT_DATA_CSV, index=False, encoding="utf-8-sig")

print("\nDataset-level V2 LIMPIO generado:")
print(" -", OUT_DATA_XLSX)
print(" -", OUT_DATA_CSV)

# -------------------------------------------------------------
# 18) RESUMEN ÚNICO PARA GRÁFICAS (1 tabla)
# metric | category | count
# -------------------------------------------------------------
rows = []

# A) Distribución niveles de madurez (según cortes nuevos 0-30-60-100)
for k, v in df["portal_maturity_level"].value_counts(dropna=False).items():
    rows.append({"metric": "PORTAL_MATURITY_LEVEL", "category": str(k), "count": int(v)})

# B) Trazabilidad (componentes)
rows.append({"metric": "TRACEABLE_ORIGEN", "category": "si", "count": int((df["traceable_origen"] == 1).sum())})
rows.append({"metric": "TRACEABLE_ORIGEN", "category": "no", "count": int((df["traceable_origen"] == 0).sum())})

rows.append({"metric": "TRACEABLE_TEMPORAL", "category": "si", "count": int((df["traceable_temporal"] == 1).sum())})
rows.append({"metric": "TRACEABLE_TEMPORAL", "category": "no", "count": int((df["traceable_temporal"] == 0).sum())})

rows.append({"metric": "TRACEABLE_REUTILIZABLE_DOI", "category": "si", "count": int((df["traceable_reutilizable"] == 1).sum())})
rows.append({"metric": "TRACEABLE_REUTILIZABLE_DOI", "category": "no", "count": int((df["traceable_reutilizable"] == 0).sum())})

# Traceability score bins (0/33/66/100)
df["traceability_score_bin"] = pd.cut(
    df["traceability_score"],
    bins=[-0.1, 1, 34, 67, 100.1],
    labels=["0", "33", "66", "100"]
).astype(str)

for k, v in df["traceability_score_bin"].value_counts(dropna=False).items():
    rows.append({"metric": "TRACEABILITY_SCORE_BIN", "category": str(k), "count": int(v)})

# C) Bins para dimensiones (usando tus cortes 0-30-60-100)
def level_0_30_60(x):
    return pd.cut(x, bins=[-0.1, 30, 60, 100], labels=["bajo", "medio", "alto"]).astype(str)

for metric_col, metric_name in [
    ("interoperability_semantics", "INTEROP_SEMANTICS_BIN"),
    ("interoperability_technical", "INTEROP_TECHNICAL_BIN"),
    ("accessibility_score", "ACCESSIBILITY_SCORE_BIN"),
    ("quality_score", "QUALITY_SCORE_BIN"),
]:
    b = level_0_30_60(pd.to_numeric(df[metric_col], errors="coerce"))
    for k, v in b.value_counts(dropna=False).items():
        rows.append({"metric": metric_name, "category": str(k), "count": int(v)})

summary = pd.DataFrame(rows)

summary.to_excel(OUT_SUM_XLSX, index=False)
summary.to_csv(OUT_SUM_CSV, index=False, encoding="utf-8-sig")

print("\nResumen único para gráficas V2 generado:")
print(" -", OUT_SUM_XLSX)
print(" -", OUT_SUM_CSV)